# Import Libruary

In [1]:
!pip uninstall -y -q tensorflow

In [2]:
import torch
torch.autograd.set_detect_anomaly(True)

print('pytorch version', torch.__version__)
print("GPU available:", torch.cuda.device_count())
#print('GPU name:',torch.cuda.get_device_name(0))

# Set the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pytorch version 2.5.1+cu124
GPU available: 0


In [3]:
from huggingface_hub import hf_hub_download

In [4]:
import transformers

# For Model
from transformers import TimeSeriesTransformerForPrediction

# For TPU
from transformers import set_seed

# Global Constants

In [5]:
MODEL_TRANSFORMER = "huggingface/time-series-transformer-tourism-monthly"

# Import Data

In [6]:
file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)

train-batch.pt:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

In [8]:
batch = torch.load(file)

<ipython-input-8-78e023422af6>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  batch = torch.load(file)


In [17]:
batch.keys()

dict_keys(['static_categorical_features', 'static_real_features', 'past_time_features', 'past_values', 'past_observed_mask', 'future_time_features', 'future_values', 'future_observed_mask'])

In [21]:
batch['static_categorical_features'].shape

torch.Size([64, 1])

In [23]:
batch['past_time_features'].shape

torch.Size([64, 61, 2])

In [26]:
batch['past_values'].shape

torch.Size([64, 61])

In [27]:
batch['past_values'].shape

torch.Size([64, 61])

In [28]:
batch['past_observed_mask'].shape

torch.Size([64, 61])

In [30]:
batch['future_time_features'].shape

torch.Size([64, 24, 2])

In [31]:
batch['future_values'].shape

torch.Size([64, 24])

In [7]:
model = TimeSeriesTransformerForPrediction.from_pretrained(MODEL_TRANSFORMER)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/151k [00:00<?, ?B/s]

# Test the model

In [9]:
outputs = model(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_values=batch["future_values"],
    future_time_features=batch["future_time_features"],
)

In [10]:
loss = outputs.loss
loss.backward()

In [11]:
outputs = model.generate(
    past_values=batch["past_values"],
    past_time_features=batch["past_time_features"],
    past_observed_mask=batch["past_observed_mask"],
    static_categorical_features=batch["static_categorical_features"],
    static_real_features=batch["static_real_features"],
    future_time_features=batch["future_time_features"],
)

In [12]:
mean_prediction = outputs.sequences.mean(dim=1)

In [13]:
mean_prediction

tensor([[25585.5527, 32823.4922, 34573.8516,  ..., 34369.9648, 36624.2383,
         28631.4043],
        [26469.1641, 36083.3242, 49398.5312,  ..., 26694.1895, 27384.5352,
         24625.1641],
        [ 1191.8763,  1079.2697,  1150.9725,  ...,  2115.2458,  1999.7454,
          1718.1730],
        ...,
        [ 2571.4785,  3734.4407,  3528.0156,  ...,  3726.3481,  4251.6445,
          4481.8052],
        [  768.2939,   569.4346,   306.9073,  ...,   704.8643,  1425.4165,
          1314.7802],
        [  384.4711,   564.0748,   395.4701,  ...,   392.4492,   369.0707,
           464.2245]])